In [11]:
import json
import requests



# Ліміт запитів
REQUEST_LIMIT = 1000
request_count = 0
request_count_google = 0
# Функція для геокодування локацій за допомогою LocationIQ
def geocode_location(location, city="Київ", country="Україна"):
    global request_count
    if request_count >= REQUEST_LIMIT:
        print(f"Request limit reached. Skipping geocoding for {location}.")
        return None, None

    try:
        query = f'{location}, {city}, {country}'
        geocoding_url = f'https://us1.locationiq.com/v1/search.php?key={LOCATIONIQ_API_KEY}&q={query}&format=json'
        
        response = requests.get(geocoding_url)
        if response.status_code == 200:
            data = response.json()
            if data:
                latitude = float(data[0]["lat"])
                longitude = float(data[0]["lon"])
                print(f"Geocoded: {location} -> Latitude: {latitude}, Longitude: {longitude}")
                request_count += 1  # Підвищуємо лічильник запитів
                # Перевірка, чи знаходиться адреса в межах Києва (широта 50.21-50.53, довгота 30.23-30.82)
                if 49.8 <= latitude <= 51.00 and 29.8 <= longitude <= 31.2:
                    return latitude, longitude
                else:
                    print(f"Location {location} is outside Kyiv. Trying Google API.")
        else:
            print(f"Failed to geocode {location} using LocationIQ. Status code: {response.status_code}")
        return None, None
    except Exception as e:
        print(f"Error geocoding location {location}: {e}")
        return None, None

# Функція для геокодування за допомогою Google API
def geocode_location_google(location, city="Київ", country="Україна"):
    try:
        query = f'{location}, {city}, {country}'
        geocoding_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={query}&key={GOOGLE_API_KEY}'
        
        response = requests.get(geocoding_url)
        if response.status_code == 200:
            data = response.json()
            if data["results"]:
                location_data = data["results"][0]["geometry"]["location"]
                latitude = float(location_data["lat"])
                longitude = float(location_data["lng"])
                print(f"Geocoded (Google API): {location} -> Latitude: {latitude}, Longitude: {longitude}")
                # Перевірка, чи знаходиться адреса в межах Києва
                if 49.8 <= latitude <= 51.00 and 29.8 <= longitude <= 31.2:
                    return latitude, longitude
                else:
                    print(f"Location {location} is outside Kyiv (even in Google API).")
        else:
            print(f"Failed to geocode {location} using Google API. Status code: {response.status_code}")
        return None, None
    except Exception as e:
        print(f"Error geocoding location with Google API {location}: {e}")
        return None, None

# Завантажуємо JSON файл
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Лічильник оброблених адрес
processed_addresses = 0

# Обробляємо записи в файлі
for entry in data:
    if entry.get("location_type") == 'exact location':
        location = entry.get('location')
        if location:
            location_str = ', '.join(location)  # Об'єднуємо масив локацій в строку
            print(f"Processing location: {location_str}")
            # Геокодуємо адресу через LocationIQ
            lat, lon = geocode_location(location_str)
            
            # Якщо адреса не знаходиться в Києві, використовуємо Google API
            if (lat is None or lon is None) and request_count < REQUEST_LIMIT:
                lat, lon = geocode_location_google(location_str)
                request_count_google += 1

            if lat and lon:
                # Додаємо широту і довготу до запису
                entry['latitude'] = lat
                entry['longitude'] = lon
                processed_addresses += 1
            else:
                print(f"Could not geocode location: {location_str}")
        else:
            print("No location found in entry.")
    else:
        print(f"Skipping entry with location_type: {entry.get('location_type')}")

# Записуємо оновлений файл
with open(updated_file_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"Оновлений файл збережено. Оброблено {processed_addresses} адрес(и). Виконано {request_count} запитів через LocationIQ. Виконано {request_count_google} запитів через Google")


Processing location: KPI Dormitory 4
Failed to geocode KPI Dormitory 4 using LocationIQ. Status code: 404
Geocoded (Google API): КПІ гуртожиток 4 -> Latitude: 50.4486105, Longitude: 30.45109369999999
Processing location: KPI Dormitory 4
Failed to geocode KPI Dormitory 4 using LocationIQ. Status code: 404
Geocoded (Google API): КПІ гуртожиток 4 -> Latitude: 50.4486105, Longitude: 30.45109369999999
Processing location: KPI Dormitory 4
Failed to geocode KPI Dormitory 4 using LocationIQ. Status code: 404
Geocoded (Google API): КПІ гуртожиток 4 -> Latitude: 50.4486105, Longitude: 30.45109369999999
Processing location: KPI Dormitory 4
Failed to geocode KPI Dormitory 4 using LocationIQ. Status code: 404
Geocoded (Google API): КПІ гуртожиток 4 -> Latitude: 50.4486105, Longitude: 30.45109369999999
Оновлений файл збережено. Оброблено 4 адрес(и). Виконано 0 запитів через LocationIQ. Виконано 4 запитів через Google
